In [3]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     -- ------------------------------------- 1.0/15.3 MB 10.1 MB/s eta 0:00:02
     ---------- ----------------------------- 3.9/15.3 MB 11.2 MB/s eta 0:00:02
     ---------------- ----------------------- 6.3/15.3 MB 11.4 MB/s eta 0:00:01
     ---------------------- ----------------- 8.7/15.3 MB 11.4 MB/s eta 0:00:01
     ---------------------------- ---------- 11.0/15.3 MB 11.5 MB/s eta 0:00:01
     ---------------------------------- ---- 13.6/15.3 MB 11.6 MB/s eta 0:00:01
     --------------------------------------  15.2/15.3 MB 11.1 MB/s eta 0:00:01
     --------------------------------------- 15.3/15.3 MB 10.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import spacy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gensim.downloader
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\threadpoolctl.py:1186: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [5]:
df=pd.read_excel("full_data (2).xlsx")

In [6]:
df

,Sentence,Заперечення,Виправдовування,Заклик,Розпалювання ворожнечі та ненависті,Приниження національної честі та гідності
0,Решение очень важное.,0,0,0,0,0
1,Почему?,0,0,0,0,0
2,Потому что Государственная Дума ратифицировала...,0,0,0,0,0
3,А дальше было решение Совета Федерации.,0,0,0,0,0
4,Вот как его прокомментировала Валентина Иванов...,0,0,0,0,0
...,...,...,...,...,...,...
19365,А у нас нехорошо.,0,0,0,0,0
19366,"А у вас нехорошо, и в Европе нехорошо.",0,0,0,0,0
19367,"Но мы не злорадствуем, мы печалимся.",0,0,0,0,0
19368,"Но наше дело право, и мы победим.",0,0,0,0,0


In [9]:
print(f'Number of duplicate entries: {df.duplicated().sum()}')

Number of duplicate entries: 964


In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.shape

(18406, 7)

In [12]:
nlp = spacy.load('ru_core_news_sm')

def preprocess_text(text):
    doc = nlp(str(text).lower()) 
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]  
    return " ".join(tokens)



In [13]:
df_proc=df
df_proc['Sentence'] = df['Sentence'].apply(preprocess_text)

In [14]:
df_proc

,Sentence,Заперечення,Виправдовування,Заклик,Розпалювання ворожнечі та ненависті,Приниження національної честі та гідності,label
0,решение важный,0,0,0,0,0,5
1,,0,0,0,0,0,5
2,государственный дума ратифицировать признать с...,0,0,0,0,0,5
3,решение совет федерация,0,0,0,0,0,5
4,прокомментировать валентина иванович матвиенко,0,0,0,0,0,5
...,...,...,...,...,...,...,...
19364,сеем пашем,0,0,0,0,0,5
19365,нехороший,0,0,0,0,0,5
19366,нехороший европа нехороший,0,0,0,0,0,5
19367,злорадствовать печалиться,0,0,0,0,0,5


In [15]:
X_train, X_test, y_train, y_test = train_test_split(df_proc['Sentence'], df_proc.iloc[:, df_proc.columns != 'Sentence'], test_size=0.2, random_state=42, stratify=df_proc.iloc[:, df_proc.columns != 'Sentence'])

In [16]:
from transformers import AutoTokenizer, AutoModel
import torch

# Завантаження RuBERT
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

In [17]:
def get_avg_w2v(text, tokenizer, model):

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return vector


X_train_w2v = np.array([get_avg_w2v(text, tokenizer, model) for text in X_train])
X_test_w2v = np.array([get_avg_w2v(text, tokenizer, model) for text in X_test])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
X_train_w2v.shape

(14724, 768)

In [19]:
unique_labels = ['Заперечення', 'Виправдовування', 'Заклик', 'Розпалювання ворожнечі та ненависті', 'Приниження національної честі та гідності', 'Просто текст']
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}

y_train_label = y_train.apply(
    lambda row: next((label_to_id[label] for label in unique_labels[:-1] if row[label] == 1), label_to_id['Просто текст']),
    axis=1
)

y_test_label = y_test.apply(
    lambda row: next((label_to_id[label] for label in unique_labels[:-1] if row[label] == 1), label_to_id['Просто текст']),
    axis=1
)

In [25]:
import pandas as pd

# Якщо X_train_w2v — це NumPy масив, конвертуємо його в DataFrame
X_train_w2v = pd.DataFrame(X_train_w2v)

# Якщо y_train_label — це NumPy масив, конвертуємо його в DataFrame
y_train_label = pd.DataFrame(y_train_label, columns=["Label"]).reset_index()  # "Label" — назва цільової колонки

# Об'єднання X_train_w2v і y_train_label
train_data = pd.concat([X_train_w2v, y_train_label], axis=1)

# Збереження об'єднаних даних у файл Excel
train_data.to_excel("train_data.xlsx", index=False)
print("Об'єднані X_train_w2v і y_train_label успішно збережені у train_data.xlsx")


Об'єднані X_train_w2v і y_train_label успішно збережені у train_data.xlsx


In [26]:
X_test_w2v = pd.DataFrame(X_test_w2v)
y_test_label = pd.DataFrame(y_test_label, columns=["Label"]).reset_index() 

test_data = pd.concat([X_test_w2v, y_test_label], axis=1)
test_data.to_excel("test_data.xlsx", index=False)
print("Об'єднані X_test_w2v і y_test_label успішно збережені у test_data.xlsx")


Об'єднані X_test_w2v і y_test_label успішно збережені у test_data.xlsx
